In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] ='/content'
!kaggle datasets download -d jillanisofttech/brain-stroke-dataset
!unzip \*.zip && rm *.zip

  0% 0.00/47.2k [00:00<?, ?B/s]
100% 47.2k/47.2k [00:00<00:00, 49.3MB/s]
Archive:  brain-stroke-dataset.zip
  inflating: brain_stroke.csv        


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import NearestNeighbors
from scipy import stats
import numpy as np

class AKNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_neighbors=4):
        self.n_neighbors = n_neighbors

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        return self

    def predict(self, X):
        y_pred = np.zeros(X.shape[0], dtype=int)
        for i in range(X.shape[0]):
            distances, indices = NearestNeighbors(n_neighbors=self.n_neighbors + 1).fit(self.X_train).kneighbors([X[i]])
            nearest_labels = self.y_train[indices[0][1:]]
            mode, _ = stats.mode(nearest_labels)
            y_pred[i] = mode[0]
        return y_pred

# Load the CSV data
data = pd.read_csv('/content/brain_stroke.csv')  # Replace 'your_data.csv' with the path to your CSV file

# Assuming 'stroke' is your target variable
X = data.drop(columns=['stroke'])  # Features
y = data['stroke']  # Target variable

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Perform one-hot encoding for categorical variables
X_train_encoded = pd.get_dummies(X_train)
X_test_encoded = pd.get_dummies(X_test)

from sklearn.neighbors import KNeighborsClassifier

# Initialize and fit the adaptive k-nearest neighbors model
aknn = KNeighborsClassifier(weights='distance')
aknn.fit(X_train_encoded, y_train)

# Make predictions on the test data
y_pred = aknn.predict(X_test_encoded)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)




Accuracy: 0.9444816053511705
Precision: 0.2222222222222222
Recall: 0.02564102564102564
F1 Score: 0.04597701149425287
